In [1]:
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer

from langchain_community.document_loaders import PyPDFLoader # 1.로드
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2.청크
from langchain.embeddings import HuggingFaceEmbeddings # 3. 임베딩 모델
from langchain.vectorstores import FAISS # 3. 벡터 저장
from langchain.retrievers import BM25Retriever, EnsembleRetriever # 4. 검색 기법
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

from transformers import pipeline

/home/jonghai/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
# 1. 문서 로드 (pdf페이지로 나눠짐)
loader = PyPDFLoader("한화생명 간편가입 시그니처 암보험(갱신형) 무배당_2055-001_002_약관_20220601_(2).pdf")
documents = loader.load()

In [4]:
# 2. 문서를 적절한 크기의 조각으로 청크 (split)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30) # 청크 중 중복되는 부분 크기
  #seprator 공백이면 공백 기준으로 청크를.. \n을 기본적으로 사용 (이거는 순차적으로 진행해줌)

chunks = text_splitter.split_documents(documents)
chunks = chunks[11:] #불필요한 청크 제외(목차 등)

# 각 청크에 id 부여
for i, chunk in enumerate(chunks):
    chunk.metadata['doc_id'] = i

In [5]:
# 3. HuggingFaceBgeEmbeddings 사용하여 벡터 임베딩 생성 (의미 검색)
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") #기본모델:all-MiniLM-L6-v2 # model_name="BAAI/bge-small-en-v1.5", "BAAI/bge-m3"지정 가능

# 3. FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(documents=chunks,
                                   embedding=embedding)

/tmp/ipykernel_81475/1925248646.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3") #기본모델:all-MiniLM-L6-v2 # model_name="BAAI/bge-small-en-v1.5", "BAAI/bge-m3"지정 가능


In [6]:
top_k = 3  # 원하는 top-k 문서 개수 설정

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})

In [7]:
# 키워드 검색
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 3

In [8]:
# 의미 검색 + 키워드 검색 (앙상블)
ensemble_retrievers = [retriever, bm25_retriever]
ensemble_retriever = EnsembleRetriever(
    retrievers=ensemble_retrievers,
    weights=[0.7, 0.3]
)

In [9]:
# 6. llm + rag 파이프라인 구성
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


# LLM 초기화
llm = ChatOllama(model="llama3.1",
                 temperature=0.8,
                 num_predict=300) # 최대로 생성할 토큰 수


# 프롬프트 작성
retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면,
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")


# 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(ensemble_retriever, combine_docs_chain)

# Rouge Metric

In [11]:
pip install korouge-score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
from korouge_score import rouge_scorer
from typing import List, Dict

def calculate_rouge_similarity(
        query: str, 
        prediction: str, 
        rouge_types: List[str] = ['rouge1', 'rouge2', 'rougeL'],
        ) -> Dict[str, float]:
    
    """
    주어진 쿼리 문장과 예측 문장 사이의 선택된 ROUGE 점수를 계산합니다.

    Args:
    query (str): 기준이 되는 쿼리 문장
    prediction (str): 유사성을 비교할 예측 문장
    rouge_types (List[str]): 계산할 ROUGE 메트릭 리스트 (기본값: ['rouge1', 'rouge2', 'rougeL'])

    Returns:
    Dict[str, float]: 선택된 ROUGE 메트릭의 F1 점수를 포함하는 딕셔너리
    """
    # 입력된 ROUGE 유형의 유효성을 검사합니다.
    valid_rouge_types = set(['rouge1', 'rouge2', 'rougeL'])
    rouge_types = [rt for rt in rouge_types if rt in valid_rouge_types]
    
    if not rouge_types:
        raise ValueError("유효한 ROUGE 유형이 제공되지 않았습니다.")

    # ROUGE scorer 객체를 초기화합니다.
    scorer = rouge_scorer.RougeScorer(rouge_types, use_stemmer=True)

    # ROUGE 점수를 계산합니다.
    scores = scorer.score(query, prediction)

    # 결과를 정리합니다.
    result = {rouge_type: scores[rouge_type].fmeasure for rouge_type in rouge_types}

    return result

In [14]:
# QA 데이터셋 로드 (200개 항목)
df_qa_test = pd.read_csv("qa_test_복사본_최종최종.csv", encoding='cp949')
print(df_qa_test.shape)
df_qa_test.head()

(200, 5)


,context,source,doc_id,question,answer
0,7 / 531 \nI. 약관 이용 안내 \n1. 보험약관이란? \n보험약관은 가입하...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,0,약관은 어떤 구성으로 이루어져 있나요?,약관은 주계약 약관과 특약(특별약관)으로 나뉩니다. 주계약 약관은 기본 계약을 포함...
1,10 / 531 \n \n5. 보험금 신청방법 \n \nSTEP 1. \n보험...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,5,보험금을 신청하려면 어떤 서류가 필요한가요?,보험금 신청 시 필요한 서류는 신청하는 보험금 종류에 따라 다릅니다. 기본적으로는 ...
2,있을 수 있으니 청구 유형별로 세부내역을 확인하시기 바랍니다. \n 구분 사망 장...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,6,보험금 지급 절차는 어떻게 이루어지나요?,보험금 지급 절차는 다음과 같습니다:\n\nSTEP 1: 보험금 지급 접수\nSTE...
3,12 / 531 \nⅡ. 약관 요약서 \n1. 보험계약의 개요 \n가입하신 보험상품...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,8,해당 암보험은 어떤 특징을 가지고 있나요?,"이 보험 상품의 주요 특징은 다음과 같습니다: 보장성보험: 사망, 질병, 상해 등의..."
4,13 / 531 \n \n \n※ 3개월동안 대상특약의 보험료가 발생하지 않는 것...,한화생명 간편가입 시그니처 암보험(갱신형) 무배당,10,보장 개시일부터 보험료를 납입해야 하는 특약에는 어떤 것이 있나요?,보장 개시일부터 특약 보험료를 납입하는 특약으로는 다음이 포함됩니다:\n\n간편가입...


In [15]:
# 예시
question = df_qa_test.iloc[0]['question']
ground_truth = df_qa_test.iloc[0]['answer']
response = rag_chain.invoke({"input": question})
prediction = response['answer']

print("Question:", question)
print("Ground Truth:", ground_truth)
print("Prediction:", prediction)

rouge_scores = calculate_rouge_similarity(ground_truth, prediction, ['rouge1'])
print(f"Rouge 점수: {rouge_scores['rouge1']:.4f}")

Question: 약관은 어떤 구성으로 이루어져 있나요?
Ground Truth: 약관은 주계약 약관과 특약(특별약관)으로 나뉩니다. 주계약 약관은 기본 계약을 포함한 공통 사항에 대한 내용을 담고 있으며, 특약은 선택 가입한 보장 등에 대한 계약 내용을 포함합니다. 또한, 어려운 보험 용어나 관련 법규에 대한 안내도 함께 제공됩니다.

Prediction: 주계약 약관 - 기본계약을 포함한 공통 사항에 대한 계약 내용입니다. 특약(특별약관 ) - 선택가입한 보장 등에 대한 계약 내용입니다.

따라서, 주계약 약관은 제1관 목적 및 용어의 정의, 제2관 보험금의 지급, 제3관 계약자의 계약 전 알릴 의무 등으로 구성되어 있으며, 특약 약관은 「제1절 공통사항」과 「제2절 개별사항」으로 구성됩니다.
Rouge 점수: 0.3171


In [16]:
# 전체 데이터셋에 대해 A/B 테스트 - 여기서는 3개만 평가 (데이터프레임으로 정리)

def rouge_evaluate_qa_dataset(df_qa_test):
    
        results = []
    
        for i in range(3):
            question = df_qa_test.iloc[i]['question']
            ground_truth = df_qa_test.iloc[i]['answer']
            response = rag_chain.invoke({"input": question})
            prediction = response['answer']

            result = calculate_rouge_similarity(prediction, ground_truth)
            results.append(result)
    
        return pd.DataFrame(results)    

df_result_rouge = rouge_evaluate_qa_dataset(df_qa_test.iloc[:3])
df_result_rouge

,rouge1,rouge2,rougeL
0,0.025974,0.000000,0.025974
1,0.212766,0.021739,0.191489
2,0.116883,0.000000,0.064935
